<a href="https://colab.research.google.com/github/Ashi743/pytorch/blob/main/pytorch_NN_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [21]:
df.drop(columns=['id','Unnamed: 32'], inplace=True)

In [ ]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [22]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['diagnosis']), df['diagnosis'], test_size=0.2, random_state=42)

In [23]:
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
encoder= LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [38]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
class MySimpleNN():
  def __init__(self, X):
    self.wieghts = torch.randn(X.shape[1], requires_grad=True)
    self.bias = torch.randn(1, requires_grad=True)


  def forward(self, X):
    z =torch.matmul(X, self.wieghts) + self.bias
    y_pred= torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):
    epsilon= 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
    loss= -torch.mean(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred))
    return loss


In [27]:
lr= 0.1
epochs= 25

In [ ]:
model= MySimpleNN(X_train_tensor)

for epoch in range(epochs):
  y_pred= model.forward(X_train_tensor)
  loss= model.loss_function(y_pred, y_train_tensor)
  loss.backward()

  with torch.no_grad():
    model.wieghts -= lr*model.wieghts.grad
    model.bias -= lr*model.bias.grad

  model.wieghts.grad.zero_()
  model.bias.grad.zero_()


  print(f"Epoch: {epoch + 1}, loss : {loss.item()}")


Epoch: 1, loss : 0.2738250494003296
Epoch: 2, loss : 0.2682647705078125
Epoch: 3, loss : 0.2630697190761566
Epoch: 4, loss : 0.2582012712955475
Epoch: 5, loss : 0.25362685322761536
Epoch: 6, loss : 0.24931782484054565
Epoch: 7, loss : 0.24524956941604614
Epoch: 8, loss : 0.24140019714832306
Epoch: 9, loss : 0.2377505600452423
Epoch: 10, loss : 0.23428359627723694
Epoch: 11, loss : 0.2309841513633728
Epoch: 12, loss : 0.22783862054347992
Epoch: 13, loss : 0.22483493387699127
Epoch: 14, loss : 0.22196221351623535
Epoch: 15, loss : 0.21921071410179138
Epoch: 16, loss : 0.216571643948555
Epoch: 17, loss : 0.21403707563877106
Epoch: 18, loss : 0.21159981191158295
Epoch: 19, loss : 0.20925341546535492
Epoch: 20, loss : 0.20699192583560944
Epoch: 21, loss : 0.20481011271476746
Epoch: 22, loss : 0.20270299911499023
Epoch: 23, loss : 0.20066618919372559
Epoch: 24, loss : 0.19869562983512878
Epoch: 25, loss : 0.1967875361442566


In [ ]:
model.bias

tensor([-0.1684], requires_grad=True)

In [ ]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.9210526347160339


In [46]:
#use of nn.moduke
import torch
import torch.nn as nn

class Model(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear =nn.Linear(in_features= num_features, out_features=1)
    self.sigmoid= nn.Sigmoid()
  def forward(self ,features):
    out= self.linear(features)
    out= self.sigmoid(out)
    #z= wx+b
    return out


In [11]:
#create dataset
features= torch.rand(10,5)
features

tensor([[0.0353, 0.9897, 0.0556, 0.0382, 0.4750],
        [0.4322, 0.3835, 0.5315, 0.9017, 0.3304],
        [0.3204, 0.2093, 0.8670, 0.0226, 0.5324],
        [0.9618, 0.8527, 0.8456, 0.7787, 0.0948],
        [0.1777, 0.7394, 0.8428, 0.7905, 0.5826],
        [0.6645, 0.2235, 0.6998, 0.8936, 0.0363],
        [0.3102, 0.7030, 0.0553, 0.9236, 0.3544],
        [0.7981, 0.6294, 0.5343, 0.7458, 0.2782],
        [0.9646, 0.8991, 0.5104, 0.5109, 0.4606],
        [0.8066, 0.6263, 0.8359, 0.8769, 0.5076]])

In [12]:
#create model
model =Model(features.shape[1]) #instance

#call model for forward pass
#model.forward(features)    A MAGIC FUCTION CREATED BY NN.MODULE TO CALL FORWARD automatically

model(features)

tensor([[0.3774],
        [0.3539],
        [0.3874],
        [0.3749],
        [0.3827],
        [0.3639],
        [0.3441],
        [0.3746],
        [0.3753],
        [0.3788]], grad_fn=<SigmoidBackward0>)

In [15]:
#show model weights
model.linear1.weight , model.linear1.bias

(Parameter containing:
 tensor([[-0.1444, -0.0833,  0.3055, -0.1486,  0.2148],
         [-0.1300, -0.4368, -0.3140,  0.2646,  0.2368],
         [-0.2867, -0.0284, -0.0046,  0.0440, -0.2698]], requires_grad=True),
 Parameter containing:
 tensor([0.4344, 0.2488, 0.1188], requires_grad=True))

In [8]:
!pip install torchinfo

In [14]:
#visualize
from torchinfo import summary
summary(model, input_size= (10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [10]:
#SOME HIDDEN LAYERS
class Model(nn.Module):
  def __init__(self, features):
    super().__init__()
    self.linear1= nn.Linear(in_features=5, out_features=3)
    self.relu= nn.ReLU()
    self.linear2= nn.Linear(in_features=3, out_features=1)
    self.sigmoid= nn.Sigmoid()

  def forward(self, features):
    out= self.linear1(features)
    out= self.relu(out)
    out= self.linear2(out)
    out= self.sigmoid(out)
    return out

In [30]:
# TO AVOID FORWARD  CUMBERSOME
class Model(nn.Module):
  def __init__(self, features):
    super().__init__()
    self.network= nn.Sequential( nn.Linear(in_features=5, out_features=3),
                                nn.ReLU(),
                                nn.Linear(in_features=3, out_features=1),
                                nn.Sigmoid()
                              )

  def forward(self, features):
    out= self.network(features)

    return out

  #no loss funcrtion now


In [17]:
loss_fn =nn.BCELoss()


In [35]:
print(type(X_train_tensor), X_train_tensor.shape)


<class 'torch.Tensor'> torch.Size([455, 30])


In [52]:
lr= 0.1
epochs= 25

In [53]:
model= Model(X_train_tensor.shape[1])
#DEFINE OPTIMIZER

optimizer= torch.optim.SGD(model.parameters() ,lr= lr)

for epoch in range(epochs):
  y_pred= model(X_train_tensor)
  #define loss
  loss= loss_fn(y_pred, y_train_tensor.view(-1,1))

  #clear gradient
  optimizer.zero_grad()

  #backward pass
  loss.backward()

  #with torch.no_grad():
   # model.linear.weight -= lr*model.linear.weight.grad
    #model.linear.bias -= lr*model.linear.bias.grad

  #params update
  optimizer.step()

  #model.linear.weight.grad.zero_()
  #model.linear.bias.grad.zero_()



  print(f"Epoch: {epoch + 1}, loss : {loss.item()}")


Epoch: 1, loss : 0.525711715221405
Epoch: 2, loss : 0.4397173225879669
Epoch: 3, loss : 0.38640618324279785
Epoch: 4, loss : 0.3496778905391693
Epoch: 5, loss : 0.3225347399711609
Epoch: 6, loss : 0.30146118998527527
Epoch: 7, loss : 0.28449416160583496
Epoch: 8, loss : 0.27044883370399475
Epoch: 9, loss : 0.2585660219192505
Epoch: 10, loss : 0.2483355849981308
Epoch: 11, loss : 0.23940140008926392
Epoch: 12, loss : 0.23150651156902313
Epoch: 13, loss : 0.2244606763124466
Epoch: 14, loss : 0.2181195467710495
Epoch: 15, loss : 0.21237146854400635
Epoch: 16, loss : 0.20712842047214508
Epoch: 17, loss : 0.20231997966766357
Epoch: 18, loss : 0.19788900017738342
Epoch: 19, loss : 0.1937885731458664
Epoch: 20, loss : 0.18997962772846222
Epoch: 21, loss : 0.18642942607402802
Epoch: 22, loss : 0.18311025202274323
Epoch: 23, loss : 0.17999838292598724
Epoch: 24, loss : 0.1770734190940857
Epoch: 25, loss : 0.17431773245334625


In [54]:
# model evaluation
with torch.no_grad():
  y_pred = model(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5624807476997375
